# Module 4: Queue Mining & Waiting Time Analysis
## NTH-ED Triage Lead Decision Support Tool

**Purpose:** Analyze waiting times and queue patterns to identify bottlenecks and optimize patient flow.

**Why This Matters for Triage Lead:**
- **Real-time Decisions:** Which zone should I send this patient to?
- **Staff Allocation:** Where do we need more resources right now?
- **Patient Satisfaction:** Long waits = LWBS risk, complaints
- **Performance Targets:** Are we meeting PIA benchmarks by triage level?

**Key Metrics:**
- PIA (Physician Initial Assessment) = Registration → Assessment
- Triage-to-Registration time
- Consult wait time
- Total Length of Stay (LOS)

---

## Cell 1: Imports

In [25]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, HTML
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

## Cell 2: Load Dependencies

In [26]:
# Run helper notebook for constants
%run ../utils/helpers.ipynb

# Run data loader notebook
%run data_loader.ipynb

NTH-ED DATA LOADING PIPELINE
📥 Loading event log...
   ✓ Loaded 90,965 events
   ✓ 16,011 unique patient visits
   ✓ Columns: ['visit_id', 'patient_id', 'initial_zone', 'age', 'month', 'day', 'gender', 'triage_code', 'triage_desc', 'disposition_code', 'disposition_desc', 'consult_desc', 'cdu_flag', 'consult_req_flag', 'consult_arrival_flag', 'event', 'timestamp']

⏰ Parsing timestamps...
   ✓ All timestamps parsed successfully
   ✓ Date range: 2021-03-31 23:59:00 to 2021-06-01 17:16:00

🔧 Handling missing data...
   • initial_zone: 1,950 missing (2.1%)
   • triage_code: 3 missing (0.0%)
   • consult_desc: 69,698 missing (76.6%)
   • age: 0 missing (0.0%)
   ✓ Missing zones filled with 'Unknown'
   ✓ Missing consults marked as 'No Consult'

📋 Standardizing columns for process mining...
   ✓ Created process mining columns: case_id, activity, resource
   ✓ Created outcome flags: is_admitted, is_lwbs

🔀 Sorting events with logical ordering...
   ✓ Events sorted by case_id → timestamp → log

## Cell 3: Load Data

In [27]:
# Load the data
filepath = "/Users/ishaandawra/Desktop/Machine Learning Notes/Machine Learning Projects/Analytics_Colloquia_Project/data/event_log_ED_MMA_2026.csv"
event_log, visits = load_and_prepare_data(filepath)

print(f"\n✓ Data loaded: {len(visits):,} patient visits")
print(f"✓ Columns with wait times: pia_minutes, triage_to_reg_minutes, consult_wait_minutes, los_minutes")

NTH-ED DATA LOADING PIPELINE
📥 Loading event log...
   ✓ Loaded 90,965 events
   ✓ 16,011 unique patient visits
   ✓ Columns: ['visit_id', 'patient_id', 'initial_zone', 'age', 'month', 'day', 'gender', 'triage_code', 'triage_desc', 'disposition_code', 'disposition_desc', 'consult_desc', 'cdu_flag', 'consult_req_flag', 'consult_arrival_flag', 'event', 'timestamp']

⏰ Parsing timestamps...
   ✓ All timestamps parsed successfully
   ✓ Date range: 2021-03-31 23:59:00 to 2021-06-01 17:16:00

🔧 Handling missing data...
   • initial_zone: 1,950 missing (2.1%)
   • triage_code: 3 missing (0.0%)
   • consult_desc: 69,698 missing (76.6%)
   • age: 0 missing (0.0%)
   ✓ Missing zones filled with 'Unknown'
   ✓ Missing consults marked as 'No Consult'

📋 Standardizing columns for process mining...
   ✓ Created process mining columns: case_id, activity, resource
   ✓ Created outcome flags: is_admitted, is_lwbs

🔀 Sorting events with logical ordering...
   ✓ Events sorted by case_id → timestamp → log

---
## Section 1: Define Wait Time Benchmarks

### Canadian Triage & Acuity Scale (CTAS) PIA Targets

| CTAS Level | Description | PIA Target |
|------------|-------------|------------|
| 1 | Resuscitation | Immediate (0 min) |
| 2 | Emergency | 15 minutes |
| 3 | Urgent | 30 minutes |
| 4 | Less Urgent | 60 minutes |
| 5 | Non-Urgent | 120 minutes |

---

## Cell 4: Define Benchmarks & Thresholds

In [28]:
# PIA targets by CTAS level (in minutes)
PIA_TARGETS = {
    '1-RESUSCITATION': 0,
    '2-EMERGENCY': 15,
    '3-URGENT': 30,
    '4-LESS URGENT': 60,
    '5-NON-URGENT': 120
}

# Alert thresholds (multiples of target)
ALERT_THRESHOLDS = {
    'warning': 1.5,   # 1.5x target = yellow alert
    'critical': 2.0   # 2x target = red alert
}

# LOS benchmarks (in minutes)
LOS_BENCHMARKS = {
    'target': 240,      # 4 hours target
    'warning': 360,     # 6 hours warning
    'critical': 480     # 8 hours critical
}

print("✓ Benchmarks defined")
print(f"  • PIA targets: {PIA_TARGETS}")
print(f"  • LOS target: {LOS_BENCHMARKS['target']} min ({LOS_BENCHMARKS['target']/60:.0f} hours)")

✓ Benchmarks defined
  • PIA targets: {'1-RESUSCITATION': 0, '2-EMERGENCY': 15, '3-URGENT': 30, '4-LESS URGENT': 60, '5-NON-URGENT': 120}
  • LOS target: 240 min (4 hours)


---
## Section 2: Overall Wait Time Statistics

---

## Cell 5: Calculate Overall Wait Time Statistics

In [29]:
def calculate_wait_time_stats(visits: pd.DataFrame, column: str) -> dict:
    """
    Calculate comprehensive statistics for a wait time column.
    
    Returns dict with mean, median, std, percentiles, etc.
    """
    data = visits[column].dropna()
    
    return {
        'count': len(data),
        'mean': data.mean(),
        'median': data.median(),
        'std': data.std(),
        'min': data.min(),
        'max': data.max(),
        'p25': data.quantile(0.25),
        'p75': data.quantile(0.75),
        'p90': data.quantile(0.90),
        'p95': data.quantile(0.95)
    }

# Calculate stats for all key wait times
wait_time_cols = ['pia_minutes', 'triage_to_reg_minutes', 'consult_wait_minutes', 'los_minutes']

print("=" * 70)
print("OVERALL WAIT TIME STATISTICS")
print("=" * 70)

overall_stats = {}
for col in wait_time_cols:
    stats = calculate_wait_time_stats(visits, col)
    overall_stats[col] = stats
    
    col_name = col.replace('_', ' ').title().replace('Pia', 'PIA').replace('Los', 'LOS')
    print(f"\n📊 {col_name}:")
    print(f"   Median: {stats['median']:.0f} min | Mean: {stats['mean']:.0f} min")
    print(f"   Range: {stats['min']:.0f} - {stats['max']:.0f} min")
    print(f"   90th percentile: {stats['p90']:.0f} min | 95th: {stats['p95']:.0f} min")

OVERALL WAIT TIME STATISTICS

📊 PIA Minutes:
   Median: 34 min | Mean: 48 min
   Range: 0 - 591 min
   90th percentile: 106 min | 95th: 132 min

📊 Triage To Reg Minutes:
   Median: 6 min | Mean: 7 min
   Range: 0 - 818 min
   90th percentile: 12 min | 95th: 15 min

📊 Consult Wait Minutes:
   Median: 115 min | Mean: 150 min
   Range: 0 - 3048 min
   90th percentile: 304 min | 95th: 385 min

📊 LOS Minutes:
   Median: 191 min | Mean: 288 min
   Range: 4 - 4012 min
   90th percentile: 600 min | 95th: 928 min


## Cell 6: PIA Performance vs Targets

In [30]:
def calculate_pia_performance(visits: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate PIA performance against CTAS targets.
    
    WHY: Triage Lead needs to know if we're meeting targets by acuity level.
    Priority inversion (lower acuity seen faster) is a RED FLAG.
    """
    
    results = []
    
    for triage_level, target in PIA_TARGETS.items():
        subset = visits[visits['triage_level'] == triage_level]['pia_minutes'].dropna()
        
        if len(subset) > 0:
            # Calculate % meeting target
            if target == 0:
                meeting_target = (subset <= 5).mean() * 100  # Within 5 min for CTAS 1
            else:
                meeting_target = (subset <= target).mean() * 100
            
            results.append({
                'Triage Level': triage_level,
                'Target (min)': target if target > 0 else 'Immediate',
                'Median PIA (min)': round(subset.median(), 1),
                'Mean PIA (min)': round(subset.mean(), 1),
                '% Meeting Target': round(meeting_target, 1),
                'Cases': len(subset)
            })
    
    return pd.DataFrame(results)

pia_performance = calculate_pia_performance(visits)

print("\n" + "=" * 70)
print("PIA PERFORMANCE VS CTAS TARGETS")
print("=" * 70)
print("\n")
display(pia_performance)


PIA PERFORMANCE VS CTAS TARGETS




,Triage Level,Target (min),Median PIA (min),Mean PIA (min),% Meeting Target,Cases
0,1-RESUSCITATION,Immediate,6.0,9.6,44.0,125
1,2-EMERGENCY,15,33.0,47.9,26.2,4734
2,3-URGENT,30,39.0,50.5,42.0,8614
3,4-LESS URGENT,60,29.0,39.2,78.5,1907
4,5-NON-URGENT,120,21.0,29.4,98.6,362


## Cell 7: Visualize PIA Performance

In [31]:
# Create PIA performance visualization
fig_pia = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Median PIA vs Target by Triage Level', '% Meeting PIA Target'),
    horizontal_spacing=0.12
)

# Colors for triage levels
triage_colors = ['#DC2626', '#F97316', '#EAB308', '#22C55E', '#3B82F6']

# Chart 1: Median PIA vs Target
fig_pia.add_trace(
    go.Bar(
        x=pia_performance['Triage Level'],
        y=pia_performance['Median PIA (min)'],
        name='Actual Median PIA',
        marker_color=triage_colors,
        text=pia_performance['Median PIA (min)'].apply(lambda x: f'{x:.0f}'),
        textposition='outside'
    ),
    row=1, col=1
)

# Add target line markers
targets = [5, 15, 30, 60, 120]  # Using 5 for CTAS 1
fig_pia.add_trace(
    go.Scatter(
        x=pia_performance['Triage Level'],
        y=targets,
        mode='markers',
        marker=dict(symbol='line-ew', size=20, color='black', line_width=3),
        name='Target'
    ),
    row=1, col=1
)

# Chart 2: % Meeting Target
fig_pia.add_trace(
    go.Bar(
        x=pia_performance['Triage Level'],
        y=pia_performance['% Meeting Target'],
        marker_color=triage_colors,
        text=pia_performance['% Meeting Target'].apply(lambda x: f'{x:.0f}%'),
        textposition='outside',
        showlegend=False
    ),
    row=1, col=2
)

# Add 90% target line
fig_pia.add_hline(y=90, line_dash="dash", line_color="red", 
                  annotation_text="90% Target", 
                  annotation_position="right",
                  row=1, col=2)

fig_pia.update_layout(
    height=500,
    title_text="<b>PIA Performance by Triage Level</b>",
    title_x=0.5,
    title_y=0.98,
    showlegend=True,
    legend=dict(
        orientation="h", 
        yanchor="bottom", 
        y=1.08,           # Move legend higher above chart
        xanchor="center",
        x=0.25            # Center legend over left chart
    ),
    margin=dict(t=120, b=80, l=60, r=60)  # More top margin for legend
)

# Update subplot titles position
fig_pia.update_annotations(font_size=13, y=1.05)

fig_pia.update_yaxes(title_text="Minutes", row=1, col=1)
fig_pia.update_yaxes(title_text="Percentage", range=[0, 115], row=1, col=2)
fig_pia.update_xaxes(tickangle=45)

fig_pia.show()

---
## Section 3: Wait Times by Zone (Bottleneck Analysis)

**Key Question:** Which zones have the longest waits?

---

## Cell 8: Calculate Wait Times by Zone

In [32]:
def calculate_zone_wait_times(visits: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate wait time statistics by zone.
    
    WHY: Helps Triage Lead decide where to send patients.
    If YZ is backed up, maybe route to GZ if clinically appropriate.
    """
    
    zone_stats = visits.groupby('initial_zone').agg({
        'case_id': 'count',
        'pia_minutes': ['median', 'mean', lambda x: x.quantile(0.90)],
        'los_minutes': ['median', 'mean'],
        'triage_to_reg_minutes': 'median',
        'is_lwbs': 'mean'
    }).round(1)
    
    # Flatten column names
    zone_stats.columns = [
        'Total Cases', 
        'Median PIA', 'Mean PIA', '90th PIA',
        'Median LOS', 'Mean LOS',
        'Median Triage-Reg',
        'LWBS Rate'
    ]
    
    zone_stats['LWBS Rate'] = (zone_stats['LWBS Rate'] * 100).round(1)
    zone_stats = zone_stats.reset_index()
    zone_stats = zone_stats.sort_values('Median PIA', ascending=False)
    
    return zone_stats

zone_wait_times = calculate_zone_wait_times(visits)

print("=" * 70)
print("WAIT TIMES BY ZONE (Sorted by Median PIA - Longest First)")
print("=" * 70)
print("\n")
display(zone_wait_times)

WAIT TIMES BY ZONE (Sorted by Median PIA - Longest First)




,initial_zone,Total Cases,Median PIA,Mean PIA,90th PIA,Median LOS,Mean LOS,Median Triage-Reg,LWBS Rate
5,Red,99,65.0,79.2,141.0,320.0,456.6,7.0,0.0
4,HH,14,63.5,71.4,115.3,1188.0,1191.9,8.0,0.0
1,Checkout,1,61.0,61.0,61.0,1517.0,1517.0,7.0,0.0
9,YZ,4232,51.0,62.6,129.0,222.0,270.1,5.0,0.0
7,SA,1078,40.0,52.2,123.0,436.0,607.5,7.0,0.0
0,A,3193,39.0,52.6,115.0,333.0,469.8,7.0,0.0
3,GZ,4112,27.0,36.1,77.0,100.0,125.1,5.0,0.0
2,EPZ,2503,25.0,36.8,86.0,128.0,176.7,5.0,0.0
8,Unknown,358,24.5,42.9,106.0,144.0,186.9,6.0,20.0
6,Resus,421,9.0,13.9,28.0,371.0,506.2,7.0,0.0


## Cell 9: Visualize Zone Bottlenecks

In [33]:
# Create zone bottleneck visualization
fig_zone = px.bar(
    zone_wait_times.sort_values('Median PIA'),
    x='Median PIA',
    y='initial_zone',
    orientation='h',
    color='Median PIA',
    color_continuous_scale='RdYlGn_r',  # Red = long wait (bad), Green = short (good)
    title='<b>Median PIA Time by Zone (Bottleneck Identification)</b>',
    text='Median PIA'
)

fig_zone.update_traces(texttemplate='%{text:.0f} min', textposition='outside')
fig_zone.update_layout(
    height=500,  # Increased height for more room
    xaxis_title='Median PIA (minutes)',
    yaxis_title='Zone',
    coloraxis_showscale=False,
    xaxis_range=[0, 85],
    margin=dict(b=100)  # More bottom margin for annotations
)

# Add vertical lines for benchmarks (without built-in annotations)
fig_zone.add_vline(x=30, line_dash="dash", line_color="orange")
fig_zone.add_vline(x=60, line_dash="dash", line_color="red")

# Add manual annotations below x-axis
fig_zone.add_annotation(
    x=30, y=-0.18,
    xref="x", yref="paper",
    text="CTAS 3<br>(30 min)",
    showarrow=False,
    font=dict(color="orange", size=11, weight="bold"),
    align="center"
)

fig_zone.add_annotation(
    x=60, y=-0.18,
    xref="x", yref="paper",
    text="CTAS 4<br>(60 min)",
    showarrow=False,
    font=dict(color="red", size=11, weight="bold"),
    align="center"
)

fig_zone.show()

## Cell 10: Zone Comparison Box Plot

In [34]:
# Box plot showing PIA distribution by zone
# Filter to main zones and reasonable PIA values
main_zones = ['Resus', 'YZ', 'GZ', 'EPZ', 'A', 'SA']
plot_data = visits[
    (visits['initial_zone'].isin(main_zones)) & 
    (visits['pia_minutes'] > 0) & 
    (visits['pia_minutes'] < 300)  # Cap at 5 hours for visualization
].copy()

fig_box = px.box(
    plot_data,
    x='initial_zone',
    y='pia_minutes',
    color='initial_zone',
    title='<b>PIA Time Distribution by Zone</b>',
    category_orders={'initial_zone': ['Resus', 'EPZ', 'GZ', 'A', 'SA', 'YZ']}
)

fig_box.update_layout(
    height=450,
    xaxis_title='Zone',
    yaxis_title='PIA Time (minutes)',
    showlegend=False
)

# Add target lines
fig_box.add_hline(y=30, line_dash="dash", line_color="orange", 
                  annotation_text="CTAS 3 Target")
fig_box.add_hline(y=60, line_dash="dash", line_color="red",
                  annotation_text="CTAS 4 Target")

fig_box.show()

---
## Section 4: Hourly & Daily Patterns (Queue Heatmaps)

**Key Question:** When are queues longest? When should we add staff?

---

## Cell 11: Calculate Hourly Wait Patterns

In [35]:
def calculate_hourly_patterns(visits: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate wait times by hour of day.
    
    WHY: Helps with staff scheduling.
    If PIA spikes at 2 PM, schedule more physicians for afternoon shift.
    """
    
    hourly = visits.groupby('arrival_hour').agg({
        'case_id': 'count',
        'pia_minutes': 'median',
        'los_minutes': 'median',
        'is_lwbs': 'mean'
    }).round(1)
    
    hourly.columns = ['Volume', 'Median PIA', 'Median LOS', 'LWBS Rate']
    hourly['LWBS Rate'] = (hourly['LWBS Rate'] * 100).round(2)
    hourly = hourly.reset_index()
    
    return hourly

hourly_patterns = calculate_hourly_patterns(visits)

print("=" * 70)
print("HOURLY WAIT TIME PATTERNS")
print("=" * 70)
print("\n")
display(hourly_patterns)

HOURLY WAIT TIME PATTERNS




,arrival_hour,Volume,Median PIA,Median LOS,LWBS Rate
0,0.0,410,27.5,155.5,0.0
1,1.0,329,38.0,181.0,0.0
2,2.0,253,65.0,210.0,0.0
3,3.0,241,65.0,262.0,10.0
4,4.0,207,55.5,223.5,0.0
5,5.0,223,52.0,272.0,0.0
6,6.0,211,40.0,256.0,0.0
7,7.0,392,28.0,217.0,0.0
8,8.0,516,38.5,217.5,0.0
9,9.0,641,35.0,199.5,0.0


## Cell 12: Hourly Volume and Wait Time Chart

In [36]:
# Create dual-axis chart: Volume + PIA by hour
fig_hourly = make_subplots(specs=[[{"secondary_y": True}]])

# Bar chart for volume
fig_hourly.add_trace(
    go.Bar(
        x=hourly_patterns['arrival_hour'],
        y=hourly_patterns['Volume'],
        name='Patient Volume',
        marker_color='#3B82F6',
        opacity=0.7
    ),
    secondary_y=False
)

# Line chart for PIA
fig_hourly.add_trace(
    go.Scatter(
        x=hourly_patterns['arrival_hour'],
        y=hourly_patterns['Median PIA'],
        name='Median PIA',
        mode='lines+markers',
        line=dict(color='#DC2626', width=3),
        marker=dict(size=8)
    ),
    secondary_y=True
)

fig_hourly.update_layout(
    title='<b>Patient Volume and Wait Times by Hour of Day</b>',
    height=450,
    xaxis_title='Hour of Day',
    legend=dict(orientation="h", yanchor="bottom", y=1.02)
)

fig_hourly.update_yaxes(title_text="Patient Volume", secondary_y=False)
fig_hourly.update_yaxes(title_text="Median PIA (minutes)", secondary_y=True)
fig_hourly.update_xaxes(tickmode='linear', tick0=0, dtick=2)

fig_hourly.show()

## Cell 13: Create Hour x Zone Heatmap

In [37]:
def create_hour_zone_heatmap(visits: pd.DataFrame) -> pd.DataFrame:
    """
    Create pivot table of median PIA by hour and zone.
    
    WHY: Identifies specific time-zone combinations with problems.
    E.g., "YZ is worst at 3 PM" → target intervention there.
    """
    
    # Filter to main zones
    main_zones = ['Resus', 'YZ', 'GZ', 'EPZ', 'A', 'SA']
    filtered = visits[visits['initial_zone'].isin(main_zones)].copy()
    
    # Create pivot table
    heatmap_data = filtered.pivot_table(
        values='pia_minutes',
        index='initial_zone',
        columns='arrival_hour',
        aggfunc='median'
    ).round(0)
    
    return heatmap_data

heatmap_data = create_hour_zone_heatmap(visits)

# Create heatmap
fig_heatmap = px.imshow(
    heatmap_data,
    labels=dict(x="Hour of Day", y="Zone", color="Median PIA (min)"),
    title='<b>Median PIA by Hour and Zone (Bottleneck Heatmap)</b>',
    color_continuous_scale='RdYlGn_r',
    aspect='auto'
)

fig_heatmap.update_layout(
    height=400,
    xaxis=dict(tickmode='linear', tick0=0, dtick=2)
)

fig_heatmap.show()

## Cell 14: Day of Week Analysis

In [38]:
# Calculate patterns by day of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

daily_patterns = visits.groupby('arrival_day').agg({
    'case_id': 'count',
    'pia_minutes': 'median',
    'los_minutes': 'median',
    'is_lwbs': 'mean'
}).round(1)

daily_patterns.columns = ['Volume', 'Median PIA', 'Median LOS', 'LWBS Rate']
daily_patterns['LWBS Rate'] = (daily_patterns['LWBS Rate'] * 100).round(2)
daily_patterns = daily_patterns.reindex(day_order)

print("=" * 70)
print("DAILY WAIT TIME PATTERNS")
print("=" * 70)
print("\n")
display(daily_patterns)

DAILY WAIT TIME PATTERNS




,Volume,Median PIA,Median LOS,LWBS Rate
arrival_day,,,,
Monday,2492,44.0,200.0,0.0
Tuesday,2109,32.0,194.0,0.0
Wednesday,2115,35.0,188.0,0.0
Thursday,2378,33.0,196.0,0.0
Friday,2331,39.0,199.0,0.0
Saturday,2269,31.0,179.0,0.0
Sunday,2316,31.0,176.0,0.0


## Cell 15: Day of Week Visualization

In [39]:
# Day of week chart
fig_daily = make_subplots(specs=[[{"secondary_y": True}]])

fig_daily.add_trace(
    go.Bar(
        x=daily_patterns.index,
        y=daily_patterns['Volume'],
        name='Patient Volume',
        marker_color='#8B5CF6',
        opacity=0.7
    ),
    secondary_y=False
)

fig_daily.add_trace(
    go.Scatter(
        x=daily_patterns.index,
        y=daily_patterns['Median PIA'],
        name='Median PIA',
        mode='lines+markers',
        line=dict(color='#DC2626', width=3),
        marker=dict(size=10)
    ),
    secondary_y=True
)

fig_daily.update_layout(
    title='<b>Patient Volume and Wait Times by Day of Week</b>',
    height=400,
    legend=dict(orientation="h", yanchor="bottom", y=1.02)
)

fig_daily.update_yaxes(title_text="Patient Volume", secondary_y=False)
fig_daily.update_yaxes(title_text="Median PIA (minutes)", secondary_y=True)

fig_daily.show()

---
## Section 5: Long Wait Patient Identification

**Key Question:** Which patients have been waiting too long?

---

## Cell 16: Identify Long Wait Patients

In [40]:
def identify_long_wait_patients(visits: pd.DataFrame, multiplier: float = 2.0) -> pd.DataFrame:
    """
    Identify patients whose PIA exceeded their CTAS target by a multiplier.
    
    WHY: These are the patients most at risk of:
    - LWBS (leaving without being seen)
    - Adverse outcomes (condition worsening)
    - Complaints
    
    Parameters:
    -----------
    multiplier : float - How many times the target to flag (default 2x)
    """
    
    # Add target column
    visits_with_target = visits.copy()
    visits_with_target['pia_target'] = visits_with_target['triage_level'].map(PIA_TARGETS)
    
    # Handle CTAS 1 (target = 0, use 5 min threshold)
    visits_with_target.loc[visits_with_target['pia_target'] == 0, 'pia_target'] = 5
    
    # Calculate how much they exceeded target
    visits_with_target['pia_ratio'] = visits_with_target['pia_minutes'] / visits_with_target['pia_target']
    visits_with_target['exceeded_by_min'] = visits_with_target['pia_minutes'] - visits_with_target['pia_target']
    
    # Flag long waits
    long_waits = visits_with_target[
        (visits_with_target['pia_ratio'] >= multiplier) & 
        (visits_with_target['pia_minutes'].notna())
    ].copy()
    
    # Sort by ratio (worst first)
    long_waits = long_waits.sort_values('pia_ratio', ascending=False)
    
    return long_waits

# Find patients with PIA > 2x target
long_wait_patients = identify_long_wait_patients(visits, multiplier=2.0)

print("=" * 70)
print(f"LONG WAIT PATIENTS (PIA > 2x Target)")
print("=" * 70)
print(f"\nTotal flagged: {len(long_wait_patients):,} patients ({len(long_wait_patients)/len(visits)*100:.1f}% of all visits)")

# Show by triage level
print("\nBreakdown by Triage Level:")
print(long_wait_patients.groupby('triage_level')['case_id'].count())

LONG WAIT PATIENTS (PIA > 2x Target)

Total flagged: 5,541 patients (34.6% of all visits)

Breakdown by Triage Level:
triage_level
1-RESUSCITATION      42
2-EMERGENCY        2581
3-URGENT           2836
4-LESS URGENT        82
Name: case_id, dtype: int64


## Cell 17: Long Wait Patient Table

In [41]:
# Display sample of long wait patients
display_cols = ['case_id', 'initial_zone', 'triage_level', 'pia_minutes', 
                'pia_target', 'pia_ratio', 'is_lwbs', 'disposition_desc']

print("\n" + "=" * 70)
print("SAMPLE: TOP 20 LONGEST RELATIVE WAITS")
print("=" * 70)
print("(Sorted by PIA ratio - how many times they exceeded their target)\n")

display(long_wait_patients[display_cols].head(20))


SAMPLE: TOP 20 LONGEST RELATIVE WAITS
(Sorted by PIA ratio - how many times they exceeded their target)



,case_id,initial_zone,triage_level,pia_minutes,pia_target,pia_ratio,is_lwbs,disposition_desc
5816,7421680,A,2-EMERGENCY,591.0,15.0,39.400000,0,"Discharge to private home, condo, apt without ..."
7513,7431882,YZ,2-EMERGENCY,509.0,15.0,33.933333,0,"Discharge to private home, condo, apt with sup..."
2,7268604,Resus,2-EMERGENCY,492.0,15.0,32.800000,0,Admit to reporting facility as inpatient to an...
8830,7439170,YZ,2-EMERGENCY,317.0,15.0,21.133333,0,"Discharge to private home, condo, apt without ..."
2979,7402482,A,2-EMERGENCY,296.0,15.0,19.733333,0,"Discharge to private home, condo, apt without ..."
8822,7439158,YZ,2-EMERGENCY,283.0,15.0,18.866667,0,"Discharge to private home, condo, apt without ..."
14260,7466661,YZ,2-EMERGENCY,276.0,15.0,18.400000,0,"Discharge to private home, condo, apt without ..."
8826,7439164,YZ,2-EMERGENCY,276.0,15.0,18.400000,0,"Discharge to private home, condo, apt without ..."
11052,7451647,YZ,2-EMERGENCY,274.0,15.0,18.266667,0,"Discharge to private home, condo, apt without ..."
14268,7466712,YZ,2-EMERGENCY,272.0,15.0,18.133333,0,"Discharge to private home, condo, apt without ..."


## Cell 18: Long Wait Distribution by Zone

In [42]:
# Which zones have the most long waits?
long_wait_by_zone = long_wait_patients.groupby('initial_zone').agg({
    'case_id': 'count',
    'pia_ratio': 'mean',
    'is_lwbs': 'mean'
}).round(2)

long_wait_by_zone.columns = ['Long Wait Count', 'Avg PIA Ratio', 'LWBS Rate']
long_wait_by_zone['LWBS Rate'] = (long_wait_by_zone['LWBS Rate'] * 100).round(1)
long_wait_by_zone = long_wait_by_zone.sort_values('Long Wait Count', ascending=False)

print("=" * 70)
print("LONG WAIT PATIENTS BY ZONE")
print("=" * 70)
print("\n")
display(long_wait_by_zone)

LONG WAIT PATIENTS BY ZONE




,Long Wait Count,Avg PIA Ratio,LWBS Rate
initial_zone,,,
YZ,2069,4.29,1.0
A,1570,4.58,1.0
GZ,665,3.15,0.0
EPZ,563,3.43,0.0
SA,461,4.39,1.0
Unknown,78,3.95,4.0
Resus,64,4.20,0.0
Red,63,4.48,0.0
HH,7,4.31,0.0


---
## Section 6: Consult Wait Time Analysis

**Key Question:** How much do consults add to patient stay?

---

## Cell 19: Consult Wait Analysis

In [43]:
# Filter to cases with consults
consult_cases = visits[visits['has_consult'] == 1].copy()

print("=" * 70)
print("CONSULT WAIT TIME ANALYSIS")
print("=" * 70)

print(f"\n📊 CONSULT OVERVIEW:")
print(f"   Total cases with consults: {len(consult_cases):,} ({len(consult_cases)/len(visits)*100:.1f}% of all visits)")
print(f"   Median consult wait: {consult_cases['consult_wait_minutes'].median():.0f} minutes")
print(f"   Mean consult wait: {consult_cases['consult_wait_minutes'].mean():.0f} minutes")
print(f"   90th percentile: {consult_cases['consult_wait_minutes'].quantile(0.9):.0f} minutes")

# Compare LOS: with vs without consult
los_with_consult = visits[visits['has_consult'] == 1]['los_minutes'].median()
los_without_consult = visits[visits['has_consult'] == 0]['los_minutes'].median()

print(f"\n📊 IMPACT ON LENGTH OF STAY:")
print(f"   Median LOS without consult: {los_without_consult:.0f} minutes ({los_without_consult/60:.1f} hours)")
print(f"   Median LOS with consult: {los_with_consult:.0f} minutes ({los_with_consult/60:.1f} hours)")
print(f"   Consult adds approximately: {los_with_consult - los_without_consult:.0f} minutes ({(los_with_consult - los_without_consult)/60:.1f} hours)")

CONSULT WAIT TIME ANALYSIS

📊 CONSULT OVERVIEW:
   Total cases with consults: 2,718 (17.0% of all visits)
   Median consult wait: 115 minutes
   Mean consult wait: 150 minutes
   90th percentile: 304 minutes

📊 IMPACT ON LENGTH OF STAY:
   Median LOS without consult: 159 minutes (2.6 hours)
   Median LOS with consult: 547 minutes (9.1 hours)
   Consult adds approximately: 388 minutes (6.5 hours)


## Cell 20: Consult Wait by Specialty

In [44]:
# Analyze consult wait by specialty
consult_by_specialty = consult_cases.groupby('consult_desc').agg({
    'case_id': 'count',
    'consult_wait_minutes': ['median', 'mean', lambda x: x.quantile(0.9)]
}).round(0)

consult_by_specialty.columns = ['Cases', 'Median Wait', 'Mean Wait', '90th Pct']
consult_by_specialty = consult_by_specialty.sort_values('Cases', ascending=False)

print("\n" + "=" * 70)
print("CONSULT WAIT BY SPECIALTY")
print("=" * 70)
print("\n")
display(consult_by_specialty.head(15))


CONSULT WAIT BY SPECIALTY




,Cases,Median Wait,Mean Wait,90th Pct
consult_desc,,,,
Internal Medicine,1624,124.0,145.0,294.0
General Surgery,313,101.0,161.0,313.0
Psychiatry,293,143.0,219.0,438.0
Paediatrics,132,52.0,75.0,161.0
Obstetrics & Gynaecology,122,67.0,99.0,182.0
Orthopaedic Surgery,100,51.0,121.0,404.0
Urology,84,88.0,145.0,365.0
Otolaryngology,16,90.0,110.0,174.0
Plastic Surgery,15,178.0,311.0,640.0


## Cell 21: Visualize Consult Impact

In [45]:
# Create LOS comparison: with vs without consult
fig_consult = go.Figure()

# Box plots for LOS comparison
fig_consult.add_trace(go.Box(
    y=visits[visits['has_consult'] == 0]['los_minutes'].clip(upper=600),
    name='Without Consult',
    marker_color='#22C55E'
))

fig_consult.add_trace(go.Box(
    y=visits[visits['has_consult'] == 1]['los_minutes'].clip(upper=600),
    name='With Consult',
    marker_color='#DC2626'
))

fig_consult.update_layout(
    title='<b>Length of Stay: With vs Without Consult</b>',
    yaxis_title='LOS (minutes)',
    height=400,
    showlegend=True
)

fig_consult.show()

---
## Section 7: Queue Mining Dashboard Summary

---

## Cell 22: Key Insights Summary

In [46]:
# Calculate key insights
worst_zone = zone_wait_times.iloc[0]  # Already sorted by Median PIA descending
best_zone = zone_wait_times.iloc[-1]
peak_hour = hourly_patterns.loc[hourly_patterns['Median PIA'].idxmax()]
peak_volume_hour = hourly_patterns.loc[hourly_patterns['Volume'].idxmax()]

print("=" * 70)
print("🔍 KEY INSIGHTS FOR TRIAGE LEAD")
print("=" * 70)

print(f"""
1. BOTTLENECK ZONES
   • Worst zone: '{worst_zone['initial_zone']}' with median PIA of {worst_zone['Median PIA']:.0f} min
   • Best zone: '{best_zone['initial_zone']}' with median PIA of {best_zone['Median PIA']:.0f} min
   • Consider routing lower-acuity patients away from {worst_zone['initial_zone']} when possible

2. PEAK TIMES
   • Highest volume: {peak_volume_hour['arrival_hour']:.0f}:00 ({peak_volume_hour['Volume']:.0f} patients)
   • Longest waits: {peak_hour['arrival_hour']:.0f}:00 (median PIA: {peak_hour['Median PIA']:.0f} min)
   • Schedule additional staff coverage around these hours

3. PIA TARGET COMPLIANCE
   • CTAS 1 (Resuscitation): {pia_performance[pia_performance['Triage Level']=='1-RESUSCITATION']['% Meeting Target'].values[0]:.0f}% meeting target
   • CTAS 2 (Emergency): {pia_performance[pia_performance['Triage Level']=='2-EMERGENCY']['% Meeting Target'].values[0]:.0f}% meeting 15-min target
   • CTAS 3 (Urgent): {pia_performance[pia_performance['Triage Level']=='3-URGENT']['% Meeting Target'].values[0]:.0f}% meeting 30-min target

4. LONG WAIT PATIENTS
   • {len(long_wait_patients):,} patients waited >2x their target ({len(long_wait_patients)/len(visits)*100:.1f}%)
   • Most common zone for long waits: {long_wait_by_zone.index[0]}
   • These patients are at high risk of LWBS

5. CONSULT IMPACT
   • Consults add ~{(los_with_consult - los_without_consult)/60:.1f} hours to patient stay
   • {len(consult_cases)/len(visits)*100:.1f}% of patients require a consult
   • Consider early consult requests for likely-to-admit patients
""")

print("=" * 70)
print("💡 RECOMMENDED ACTIONS")
print("=" * 70)
print(f"""
   • Add physician coverage at {peak_hour['arrival_hour']:.0f}:00 when waits are longest
   • Monitor {worst_zone['initial_zone']} zone closely — consider overflow protocols
   • Flag patients approaching 2x PIA target for expedited assessment
   • Request consults earlier for CTAS 2-3 patients likely to be admitted
""")

🔍 KEY INSIGHTS FOR TRIAGE LEAD

1. BOTTLENECK ZONES
   • Worst zone: 'Red' with median PIA of 65 min
   • Best zone: 'Resus' with median PIA of 9 min
   • Consider routing lower-acuity patients away from Red when possible

2. PEAK TIMES
   • Highest volume: 14:00 (1069 patients)
   • Longest waits: 2:00 (median PIA: 65 min)
   • Schedule additional staff coverage around these hours

3. PIA TARGET COMPLIANCE
   • CTAS 1 (Resuscitation): 44% meeting target
   • CTAS 2 (Emergency): 26% meeting 15-min target
   • CTAS 3 (Urgent): 42% meeting 30-min target

4. LONG WAIT PATIENTS
   • 5,541 patients waited >2x their target (34.6%)
   • Most common zone for long waits: YZ
   • These patients are at high risk of LWBS

5. CONSULT IMPACT
   • Consults add ~6.5 hours to patient stay
   • 17.0% of patients require a consult
   • Consider early consult requests for likely-to-admit patients

💡 RECOMMENDED ACTIONS

   • Add physician coverage at 2:00 when waits are longest
   • Monitor Red zone close

## Cell 23: Function Reference

In [47]:
QUEUE_MINING_FUNCTIONS = """
=================================================================
MODULE 4: QUEUE MINING - FUNCTION REFERENCE
=================================================================

CORE FUNCTIONS:
---------------
1. calculate_wait_time_stats(visits, column)
   → Returns dict with mean, median, percentiles for a wait time column

2. calculate_pia_performance(visits)
   → Returns DataFrame comparing actual PIA to CTAS targets

3. calculate_zone_wait_times(visits)
   → Returns DataFrame with wait time stats by zone

4. calculate_hourly_patterns(visits)
   → Returns DataFrame with volume and wait times by hour

5. create_hour_zone_heatmap(visits)
   → Returns pivot table for heatmap visualization

6. identify_long_wait_patients(visits, multiplier)
   → Returns DataFrame of patients exceeding target by multiplier

CONSTANTS:
----------
- PIA_TARGETS: CTAS target times in minutes
- ALERT_THRESHOLDS: Warning and critical multipliers
- LOS_BENCHMARKS: Target, warning, critical LOS times

=================================================================
"""

print(QUEUE_MINING_FUNCTIONS)


MODULE 4: QUEUE MINING - FUNCTION REFERENCE

CORE FUNCTIONS:
---------------
1. calculate_wait_time_stats(visits, column)
   → Returns dict with mean, median, percentiles for a wait time column

2. calculate_pia_performance(visits)
   → Returns DataFrame comparing actual PIA to CTAS targets

3. calculate_zone_wait_times(visits)
   → Returns DataFrame with wait time stats by zone

4. calculate_hourly_patterns(visits)
   → Returns DataFrame with volume and wait times by hour

5. create_hour_zone_heatmap(visits)
   → Returns pivot table for heatmap visualization

6. identify_long_wait_patients(visits, multiplier)
   → Returns DataFrame of patients exceeding target by multiplier

CONSTANTS:
----------
- PIA_TARGETS: CTAS target times in minutes
- ALERT_THRESHOLDS: Warning and critical multipliers
- LOS_BENCHMARKS: Target, warning, critical LOS times




In [48]:
# Cell 24: Key Insights Summary

print("=" * 70)
print("🔍 KEY INSIGHTS FROM QUEUE MINING")
print("=" * 70)

# Calculate key metrics
overall_median_pia = visits['pia_minutes'].median()
overall_median_los = visits['los_minutes'].median()

# Worst and best zones
worst_zone = zone_wait_times.iloc[0]
best_zone = zone_wait_times.iloc[-1]

# Peak hours
peak_wait_hour = hourly_patterns.loc[hourly_patterns['Median PIA'].idxmax()]
peak_volume_hour = hourly_patterns.loc[hourly_patterns['Volume'].idxmax()]
lowest_wait_hour = hourly_patterns.loc[hourly_patterns['Median PIA'].idxmin()]

# PIA target compliance
ctas2_compliance = pia_performance[pia_performance['Triage Level']=='2-EMERGENCY']['% Meeting Target'].values[0]
ctas3_compliance = pia_performance[pia_performance['Triage Level']=='3-URGENT']['% Meeting Target'].values[0]

# Long wait stats
long_wait_count = len(long_wait_patients)
long_wait_pct = long_wait_count / len(visits) * 100
long_wait_lwbs_rate = long_wait_patients['is_lwbs'].mean() * 100

# Consult impact
consult_pct = len(consult_cases) / len(visits) * 100
consult_added_time = los_with_consult - los_without_consult

print(f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. OVERALL PERFORMANCE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   • Median PIA (all patients): {overall_median_pia:.0f} minutes
   • Median Length of Stay: {overall_median_los:.0f} minutes ({overall_median_los/60:.1f} hours)
   • Total patients analyzed: {len(visits):,}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
2. BOTTLENECK ZONES (Action Required)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   🔴 WORST: '{worst_zone['initial_zone']}' zone — Median PIA: {worst_zone['Median PIA']:.0f} min
   🟢 BEST:  '{best_zone['initial_zone']}' zone — Median PIA: {best_zone['Median PIA']:.0f} min
   
   ⚠️  '{worst_zone['initial_zone']}' takes {worst_zone['Median PIA'] - best_zone['Median PIA']:.0f} min longer than '{best_zone['initial_zone']}'
   → Consider load balancing or additional staffing in {worst_zone['initial_zone']}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
3. PEAK TIME PATTERNS (Staff Scheduling)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   📈 Highest Volume:    {peak_volume_hour['arrival_hour']:.0f}:00 ({peak_volume_hour['Volume']:.0f} patients)
   ⏱️  Longest Waits:     {peak_wait_hour['arrival_hour']:.0f}:00 (Median PIA: {peak_wait_hour['Median PIA']:.0f} min)
   ✅ Shortest Waits:    {lowest_wait_hour['arrival_hour']:.0f}:00 (Median PIA: {lowest_wait_hour['Median PIA']:.0f} min)
   
   → Schedule extra physicians at {peak_wait_hour['arrival_hour']:.0f}:00 to reduce wait times

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
4. PIA TARGET COMPLIANCE (Quality Metrics)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   • CTAS 2 (Emergency, 15-min target):  {ctas2_compliance:.0f}% compliant ❌ (Target: 90%)
   • CTAS 3 (Urgent, 30-min target):     {ctas3_compliance:.0f}% compliant ❌ (Target: 90%)
   
   ⚠️  CRITICAL: Only {ctas2_compliance:.0f}% of EMERGENCY patients seen within 15 minutes
   → This is a patient safety concern requiring immediate attention

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
5. LONG WAIT PATIENTS (LWBS Risk)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   • Patients waiting >2x their target: {long_wait_count:,} ({long_wait_pct:.1f}%)
   • LWBS rate among long-wait patients: {long_wait_lwbs_rate:.1f}%
   • Top zone for long waits: {long_wait_by_zone.index[0]}
   
   → Proactively check on patients approaching 2x target to prevent LWBS

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
6. CONSULT IMPACT (Capacity Planning)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   • {consult_pct:.1f}% of patients require a consult
   • Consults add ~{consult_added_time:.0f} minutes ({consult_added_time/60:.1f} hours) to LOS
   • Median consult wait: {consult_cases['consult_wait_minutes'].median():.0f} minutes
   
   → Request consults earlier for high-acuity patients likely to be admitted
""")

print("=" * 70)
print("💡 TOP 3 RECOMMENDED ACTIONS FOR TRIAGE LEAD")
print("=" * 70)
print(f"""
   1. ADD STAFF at {peak_wait_hour['arrival_hour']:.0f}:00 — this is when waits are longest
   
   2. MONITOR '{worst_zone['initial_zone']}' ZONE — {worst_zone['Median PIA']:.0f} min median PIA is unacceptable;
      consider overflow protocols or patient redirection
   
   3. FLAG CTAS 2 PATIENTS waiting >15 min — only {ctas2_compliance:.0f}% compliance
      is a critical safety gap
""")
print("=" * 70)

🔍 KEY INSIGHTS FROM QUEUE MINING

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. OVERALL PERFORMANCE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   • Median PIA (all patients): 34 minutes
   • Median Length of Stay: 191 minutes (3.2 hours)
   • Total patients analyzed: 16,011

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
2. BOTTLENECK ZONES (Action Required)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   🔴 WORST: 'Red' zone — Median PIA: 65 min
   🟢 BEST:  'Resus' zone — Median PIA: 9 min

   ⚠️  'Red' takes 56 min longer than 'Resus'
   → Consider load balancing or additional staffing in Red

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
3. PEAK TIME PATTERNS (Staff Scheduling)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
   📈 Highest Volume:    14:00 (1069 patients)
   ⏱️  Longest Waits:     2:00 (Median PIA: 65 min)
   ✅ Shorte